## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tumpat,MY,6.1978,102.1710,78.78,83,20,2.30,few clouds
1,1,Constitucion,CL,-35.3333,-72.4167,63.48,65,18,2.37,few clouds
2,2,Wellington,NZ,-41.2866,174.7756,50.58,97,100,27.63,light rain
3,3,Saint-Philippe,RE,-21.3585,55.7679,67.59,78,44,11.72,scattered clouds
4,4,Tuatapere,NZ,-46.1333,167.6833,35.46,94,45,3.85,scattered clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_input = float(input("What is the minimum temperature you would like for your trip? "))
max_input = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_input) & \
                                      (city_data_df["Max Temp"] >= min_input)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Kapaa,US,22.0752,-159.3190,81.48,87,20,5.75,few clouds
18,18,San Pedro,PH,14.3500,121.0167,81.63,79,20,2.30,few clouds
29,29,Butaritari,KI,3.0707,172.7902,81.70,75,100,15.14,light rain
36,36,Atar,MR,20.5169,-13.0499,87.15,23,0,9.55,clear sky
47,47,Hilo,US,19.7297,-155.0900,86.29,58,0,4.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
684,684,Mbour,SN,14.4220,-16.9638,83.25,74,18,14.18,few clouds
687,687,Otavi,NaN,-19.6500,17.3333,80.83,15,2,8.41,clear sky
690,690,Nueva Rosita,MX,27.9500,-101.2167,85.60,36,0,12.15,clear sky
691,691,Sur,OM,22.5667,59.5289,81.45,76,5,14.54,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                118
City                   118
Country                116
Lat                    118
Lng                    118
Max Temp               118
Humidity               118
Cloudiness             118
Wind Speed             118
Current Description    118
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_df = preferred_cities_df.dropna()
clean_travel_df.count()

City_ID                116
City                   116
Country                116
Lat                    116
Lng                    116
Max Temp               116
Humidity               116
Cloudiness             116
Wind Speed             116
Current Description    116
dtype: int64

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Kapaa,US,81.48,few clouds,22.0752,-159.3190,
18,San Pedro,PH,81.63,few clouds,14.3500,121.0167,
29,Butaritari,KI,81.70,light rain,3.0707,172.7902,
36,Atar,MR,87.15,clear sky,20.5169,-13.0499,
47,Hilo,US,86.29,clear sky,19.7297,-155.0900,
56,Victoria,HK,83.32,clear sky,22.2855,114.1577,
73,Jaisalmer,IN,85.24,clear sky,26.9147,70.9181,
74,Santa Cruz Del Norte,CU,80.83,overcast clouds,23.1539,-81.9236,
89,Seondha,IN,81.43,clear sky,26.1578,78.7797,
95,Parana,AR,82.54,clear sky,-31.7320,-60.5238,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": ""
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = str(lat) + ',' + str(lng)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Kapaa,US,81.48,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,San Pedro,PH,81.63,few clouds,14.3500,121.0167,Palines Apartment and Guesthouse
29,Butaritari,KI,81.70,light rain,3.0707,172.7902,Isles Sunset Lodge
36,Atar,MR,87.15,clear sky,20.5169,-13.0499,Oumou Elghoura
47,Hilo,US,86.29,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
671,Bardiyah,LY,81.95,clear sky,31.7561,25.0865,بورتو بردية
684,Mbour,SN,83.25,few clouds,14.4220,-16.9638,Tama Lodge
690,Nueva Rosita,MX,85.60,clear sky,27.9500,-101.2167,Hotel Rosa De Oro
691,Sur,OM,81.45,clear sky,22.5667,59.5289,Sur Plaza Hotel


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
weather_description = clean_hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))